# Daily Model Deployment

1. Load Model from S3
2. Query past 10 mintues of data from MongoDB
3. Build Features
4. Make Predictions
5. Log predictions to database

In [1]:
############### Initialize ###################

# Basics
from pymongo import MongoClient
import os
import numpy as np
import pandas as pd
import time
import boto3
import io
import warnings
warnings.filterwarnings('ignore')
import time
from datetime import date, datetime, timedelta
import subprocess


# NLP
import nltk
import spacy
spacy.load('en')
from nltk.corpus import stopwords
import preprocessor as p

# Model Infrastructure
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.pipeline import make_pipeline, Pipeline, FeatureUnion
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn import metrics
import dill as pickle

# Models
from sklearn.linear_model import LogisticRegression

# Database Setup
import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sqlalchemy import MetaData
from sqlalchemy import Table
from sqlalchemy import Column
from sqlalchemy import Integer, String, DateTime, Float

In [2]:
########## Database Setup #################
User = os.environ['DB_USER']
password = os.environ['DB_PWD']
dbname = os.environ['DB_NAME']
IP = os.environ['IP']

engine = create_engine('mysql+mysqlconnector://{}:{}@{}:3306/{}'.format(User,
                                                                        password, IP, dbname), echo=False)
conn = engine.connect()

# Check to see if the tables are created and if not, create them
meta = MetaData(engine)

# Create prediction table
if not engine.dialect.has_table(engine, 'daily_model_predictions'):
    print('Daily_Model_predictions Table does not exist')
    print('Daily_Model_predictions Table being created....')
    # Time, Source, Current Count, Count Diff
    t1 = Table('daily_model_predictions', meta,
               Column('run_time', DateTime, default=datetime.utcnow),
               Column('model_name', String(30)),
               Column('model_version_number', Integer),
               Column('Company', String(30)),
               Column('Prediction', Integer))
    t1.create()
else:
    print('Daily_Model_predictions Table Exists')

# Create table object
meta = MetaData(engine, reflect=True)
daily_model_predictions_table = meta.tables['daily_model_predictions']    

# Write Function
def database_log(name, version_number, company, prediction):
    #Need to log these items to a database.
        
    ins = daily_model_predictions_table.insert().values(
            run_time = datetime.now(),
            model_name = name,
            model_version_number = version_number,
            Company = company,
            Prediction = prediction
               )
    conn.execute(ins)

Daily_Model_predictions Table Exists


In [3]:
# Download the Model
subprocess.run(['aws', 's3','cp','s3://brandyn-twitter-sentiment-analysis/Models/Daily_Stock_Prediction_latest.pk','./Models'])

CompletedProcess(args=['aws', 's3', 'cp', 's3://brandyn-twitter-sentiment-analysis/Models/Daily_Stock_Prediction_latest.pk', './Models'], returncode=0)

In [4]:
# Load Model
### Validate Pickle ###
filename = 'Daily_Stock_Prediction_latest.pk'

with open('./Models/'+filename, 'rb') as f:
    model = pickle.load(f)

In [5]:
###################### Bring In Data #######################
#Setup Mongo and create the database and collection
User = os.environ['MONGODB_USER']
password = os.environ['MONGODB_PASS']
IP = os.environ['IP']

client = MongoClient(IP, username=User, password=password)
db = client['stock_tweets']

#Grab references
twitter_coll_reference = db.twitter
iex_coll_reference = db.iex

# Create Time bound
ten_min_bound =  pd.to_datetime(datetime.utcnow() - timedelta(hours = 48))

In [6]:
###################### Build Twitter Data Frames #####################

def get_twitter_data():
    # Create Data Frame from Mongo DB
    twitter_data = pd.DataFrame(list(twitter_coll_reference.find()))

    # Take a subset of the data, dont need all points to convert and this greatly speeds up
    twitter_data_subset = twitter_data.tail(400)

    # Need to convert the created_at to a time stamp and set to index
    twitter_data_subset['created_at'] = pd.to_datetime(twitter_data_subset['created_at'])
    twitter_data_subset.index=twitter_data_subset['created_at']

    # Create time bounded dataframe
    twitter_data = twitter_data_subset[twitter_data_subset['created_at'] >= ten_min_bound]

    # Delimited the Company List into separate rows
    delimited_twitter_data=[]

    for item in twitter_data.itertuples():
        #twitter_dict={}
        for company in item[1]:
            twitter_dict={}
            twitter_dict['created_at']=item[0]
            twitter_dict['company']=company
            twitter_dict['text']=item[11]
            twitter_dict['user_followers_count']=item[12]
            twitter_dict['user_name']=item[13]
            twitter_dict['user_statuses_count']=item[15]
            delimited_twitter_data.append(twitter_dict)

    delimited_twitter_df = pd.DataFrame(delimited_twitter_data) 
    delimited_twitter_df.set_index('created_at', inplace=True)

    # Create hourly data frame
    twitter_delimited_daily = delimited_twitter_df.groupby([pd.Grouper(freq="D"), 'company']).count()['text'].to_frame()
    twitter_delimited_daily.columns = ['Number_of_Tweets']

    # Concatenate the text with a space to not combine words.
    twitter_delimited_daily['text']=delimited_twitter_df.groupby([pd.Grouper(freq="D"), 'company'])['text'].apply(lambda x: ' '.join(x))
    # Number of Users
    twitter_delimited_daily['Number_of_Users'] = delimited_twitter_df.groupby([pd.Grouper(freq="D"), 'company'])['user_name'].nunique()

    # Rename Index
    twitter_delimited_daily = twitter_delimited_daily.reindex(twitter_delimited_daily.index.rename(['Time', 'Company']))
    return twitter_delimited_daily

In [7]:
##################### Build Stock Data Frames ###########################

def get_stock_data():
    stock_data = pd.DataFrame(list(iex_coll_reference.find()))
    
    # Take a subset of the data, dont need all points to convert and this greatly speeds up
    stock_data_subset = stock_data.tail(400)
    
    # Need to convert the created_at to a time stamp
    stock_data_subset['latestUpdate'] = pd.to_datetime(stock_data_subset['latestUpdate'])
    stock_data_subset.index=stock_data_subset['latestUpdate']
    
    # Create time bounded dataframe
    stock_data = stock_data_subset[stock_data_subset['latestUpdate'] >= ten_min_bound]
    
    # Create delimited dataframe
    stock_delimited_daily = stock_data.groupby([pd.Grouper(freq="D"), 'Ticker'])['latestVolume'].mean().to_frame()
    stock_delimited_daily.columns = ['Mean_Volume']
    
    # Rename the Index
    stock_delimited_daily = stock_delimited_daily.reindex(stock_delimited_daily.index.rename(['Time', 'Company']))
    return stock_delimited_daily
    

In [8]:
# Call Functions
twitter_delimited_daily = get_twitter_data()
stock_delimited_daily = get_stock_data()

# Combine Dataframes
daily_df = pd.concat([twitter_delimited_daily, stock_delimited_daily], axis=1, join='inner')

# To flatten after combined everything. 
daily_df.reset_index(inplace=True)
daily_df.head()

,Time,Company,Number_of_Tweets,text,Number_of_Users,Mean_Volume
0,2018-05-18,AAPL,31,Step by Step Dividend Investing: A Beginner's ...,25,1.366410e+07
1,2018-05-18,AMZN,41,Helluvah specific price there. Four decimal p...,37,1.285560e+06
2,2018-05-18,BA,52,5.18.18 Elliott Wave Updates For Momentum: $BA...,43,3.043739e+06
3,2018-05-18,BABA,12,5.18.18 Elliott Wave Updates For Momentum: $BA...,12,7.864113e+06
4,2018-05-18,BAC,10,#DivestFromWar\n\nDivest from\nBank of America...,9,4.648342e+07


In [9]:
# Clean the Tweets
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.RESERVED, p.OPT.EMOJI, p.OPT.HASHTAG)
def preprocess_tweet(tweet):
    return p.clean(tweet)

# Clean the tweets, by removing special characters
start_time = time.time()
daily_df['Clean_text'] = daily_df['text'].apply(lambda x: preprocess_tweet(x))

# Split Between Outcome and Features
features = daily_df[['Number_of_Tweets', 'Number_of_Users','Mean_Volume','Clean_text']]

In [18]:
# Create lemmatizer using spacy
lemmatizer = spacy.lang.en.English()

# Define Model
lr_model = LogisticRegression(n_jobs=5, penalty='l2', class_weight='balanced', solver='newton-cg', C=10)

# Define custom Tokenizer
def custom_tokenizer(doc):
    tokens = lemmatizer(doc)
    return([token.lemma_ for token in tokens if not token.is_punct])

# Define Vectorizer
vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer, stop_words='english',
                             lowercase=True, use_idf=True, max_df=2,
                             min_df=2, norm='l2', smooth_idf=True, ngram_range=(1, 2))



# Predictions
y = model.predict(features)

In [13]:
daily_df['predictions'] = y

In [14]:
daily_df

,Time,Company,Number_of_Tweets,text,Number_of_Users,Mean_Volume,Clean_text,predictions
0,2018-05-18,AAPL,31,Step by Step Dividend Investing: A Beginner's ...,25,1.366410e+07,Step by Step Dividend Investing: A Beginner's ...,0
1,2018-05-18,AMZN,41,Helluvah specific price there. Four decimal p...,37,1.285560e+06,Helluvah specific price there. Four decimal po...,0
2,2018-05-18,BA,52,5.18.18 Elliott Wave Updates For Momentum: $BA...,43,3.043739e+06,5.18.18 Elliott Wave Updates For Momentum: $BA...,0
3,2018-05-18,BABA,12,5.18.18 Elliott Wave Updates For Momentum: $BA...,12,7.864113e+06,5.18.18 Elliott Wave Updates For Momentum: $BA...,0
4,2018-05-18,BAC,10,#DivestFromWar\n\nDivest from\nBank of America...,9,4.648342e+07,Divest from Bank of America $BAC Citigroup $C ...,0
5,2018-05-18,C,47,Mastrapasqua Asset Management Upped Its Cisco ...,33,1.297626e+07,Mastrapasqua Asset Management Upped Its Cisco ...,0
6,2018-05-18,CMCSA,5,Financial Counselors Has Cut By $2.55 Million ...,5,1.200967e+07,Financial Counselors Has Cut By $2.55 Million ...,1
7,2018-05-18,CSCO,8,Mastrapasqua Asset Management Upped Its Cisco ...,7,1.808259e+07,Mastrapasqua Asset Management Upped Its Cisco ...,0
8,2018-05-18,FB,32,Socially responsible funds dump or rethink Fac...,28,7.895356e+06,Socially responsible funds dump or rethink Fac...,1
9,2018-05-18,GE,19,@tapenal @OptionsHawk profit going into more $...,12,2.653425e+07,profit going into more $GE . General Electric ...,0
